In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [2]:
def clean_df():
    path = f'{file_dir}/Resources/first_round_csvs'
    all_files = glob.glob(os.path.join(path + "/*.csv"))

    li = []

    for filename in all_files:
        df = pd.read_csv(filename,encoding_errors='ignore', index_col=None, header=0)
        li.append(df)

    first_round_df = pd.concat(li, axis=0, ignore_index=True)
    first_round_df = first_round_df.sort_values(by=['Year', 'OvPck']).reset_index()
    first_round_df = first_round_df.drop(['index', 'DT', 'FrRnd', 'RdPck'], axis=1)
    
    first_round_df['Tm'] = first_round_df['Tm'].str.replace(r'via(.*$)', '', regex=True)
    first_round_df['Name'] = first_round_df['Name'].str.replace(r'\((.+?)\)', '', regex=True) 
    first_round_df['Type'] = first_round_df['Type'].fillna('None')
    first_round_df['Type'] = first_round_df['Type'].fillna('None')
    
    first_round_df['WAR'] = first_round_df['WAR'].fillna(0)
    first_round_df['G'] = first_round_df['G'].fillna(0)
    first_round_df['AB'] = first_round_df['AB'].fillna(0)
    first_round_df['HR'] = first_round_df['HR'].fillna(0)
    first_round_df['BA'] = first_round_df['BA'].fillna(0)
    first_round_df['OPS'] = first_round_df['OPS'].fillna(0)
    first_round_df['G.1'] = first_round_df['G.1'].fillna(0)
    first_round_df['W'] = first_round_df['W'].fillna(0)
    first_round_df['L'] = first_round_df['L'].fillna(0)
    first_round_df['ERA'] = first_round_df['ERA'].fillna(0)
    first_round_df['WHIP'] = first_round_df['WHIP'].fillna(0)
    first_round_df['SV'] = first_round_df['SV'].fillna(0)
    
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return 0
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            s = re.sub('\$|,','', s)
            value = float(s)
            return value
        else:
            return 0
    
    def parse_rnd(s):
        if type(s) != str:
            return int(s)
        elif re.match('1s', s):
            s = re.sub('s', '', s)
            value = int(s)
            return value
        else:
            return int(s)    
    
    first_round_df['Bonus'] = first_round_df['Bonus'].apply(parse_dollars)
    first_round_df['Rnd'] = first_round_df['Rnd'].apply(parse_rnd)
    
    return first_round_df

In [3]:
file_dir = 'C:/Users/josha/Final_Project_Baseball_draft_prediction/'
player_file = f'{file_dir}/Resources/first_round_csvs'

In [4]:
player_file = clean_df()
first_round_df = player_file
first_round_df.head()

,Year,Rnd,OvPck,Tm,Signed,Bonus,Name,Pos,WAR,G,...,OPS,G.1,W,L,ERA,WHIP,SV,Type,Drafted Out of,Team Made Playoffs
0,2000,1,1,Marlins,Y,3000000.0,Adrian Gonzalez,1B,43.5,1929.0,...,0.843,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Eastlake HS (Chula Vista, CA)",0
1,2000,1,2,Twins,Y,2500000.0,Adam Johnson,RHP,-1.1,1.0,...,0.000,9.0,1.0,3.0,10.25,2.05,0.0,4Yr,"California State University, Fullerton (Fuller...",0
2,2000,1,3,Cubs,Y,2750000.0,Luis Montanez,SS,-1.3,129.0,...,0.586,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Coral Park HS (Miami, FL)",0
3,2000,1,4,Royals,Y,2500000.0,Mike Stodolka,LHP,0.0,0.0,...,0.000,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Centennial HS (Corona, CA)",0
4,2000,1,5,Expos,Y,2950000.0,Justin Wayne,RHP,-1.5,26.0,...,0.000,26.0,5.0,8.0,6.13,1.65,0.0,4Yr,"Stanford University (Palo Alto, CA)",0


In [5]:
# Update Column Headers
first_round_df = first_round_df.rename(columns={'Rnd': 'Round', 'OvPck': 'Overall_Pick', 'Tm': 'Team', 'Pos': 'Position', 'Type': 'Draft_Type', 'G.1': 'G1', 'Drafted Out of': 'Drafted_Out_Of', 'Team Made Playoffs': 'Team_Made_Playoffs' })

In [6]:
first_round_df.head()

,Year,Round,Overall_Pick,Team,Signed,Bonus,Name,Position,WAR,G,...,OPS,G1,W,L,ERA,WHIP,SV,Draft_Type,Drafted_Out_Of,Team_Made_Playoffs
0,2000,1,1,Marlins,Y,3000000.0,Adrian Gonzalez,1B,43.5,1929.0,...,0.843,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Eastlake HS (Chula Vista, CA)",0
1,2000,1,2,Twins,Y,2500000.0,Adam Johnson,RHP,-1.1,1.0,...,0.000,9.0,1.0,3.0,10.25,2.05,0.0,4Yr,"California State University, Fullerton (Fuller...",0
2,2000,1,3,Cubs,Y,2750000.0,Luis Montanez,SS,-1.3,129.0,...,0.586,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Coral Park HS (Miami, FL)",0
3,2000,1,4,Royals,Y,2500000.0,Mike Stodolka,LHP,0.0,0.0,...,0.000,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Centennial HS (Corona, CA)",0
4,2000,1,5,Expos,Y,2950000.0,Justin Wayne,RHP,-1.5,26.0,...,0.000,26.0,5.0,8.0,6.13,1.65,0.0,4Yr,"Stanford University (Palo Alto, CA)",0


In [7]:
# Saving cleaned data
#file_path = "Resources/Data_Clean_For_Html.csv"
#first_round_df.to_csv(file_path, index=False)

In [8]:
# put dataframe into dictionary 
first_round_dict = first_round_df.to_dict('records')

In [9]:
first_round_dict

[{'Year': 2000,
  'Round': 1,
  'Overall_Pick': 1,
  'Team': 'Marlins',
  'Signed': 'Y',
  'Bonus': 3000000.0,
  'Name': 'Adrian Gonzalez',
  'Position': '1B',
  'WAR': 43.5,
  'G': 1929.0,
  'AB': 7139.0,
  'HR': 317.0,
  'BA': 0.287,
  'OPS': 0.843,
  'G1': 0.0,
  'W': 0.0,
  'L': 0.0,
  'ERA': 0.0,
  'WHIP': 0.0,
  'SV': 0.0,
  'Draft_Type': 'HS',
  'Drafted_Out_Of': 'Eastlake HS (Chula Vista, CA)',
  'Team_Made_Playoffs': 0},
 {'Year': 2000,
  'Round': 1,
  'Overall_Pick': 2,
  'Team': 'Twins',
  'Signed': 'Y',
  'Bonus': 2500000.0,
  'Name': 'Adam Johnson',
  'Position': 'RHP',
  'WAR': -1.1,
  'G': 1.0,
  'AB': 2.0,
  'HR': 0.0,
  'BA': 0.0,
  'OPS': 0.0,
  'G1': 9.0,
  'W': 1.0,
  'L': 3.0,
  'ERA': 10.25,
  'WHIP': 2.05,
  'SV': 0.0,
  'Draft_Type': '4Yr',
  'Drafted_Out_Of': 'California State University, Fullerton (Fullerton, CA)',
  'Team_Made_Playoffs': 0},
 {'Year': 2000,
  'Round': 1,
  'Overall_Pick': 3,
  'Team': 'Cubs',
  'Signed': 'Y',
  'Bonus': 2750000.0,
  'Name': '

In [11]:
import json
with open('html_data.js', 'w') as out_file:
  out_file.write('var data = %s;' % json.dumps(first_round_dict))

In [9]:
data = first_round_dict

In [10]:
import json


In [11]:
first_round_json = json.dumps(data, indent=2)
print(first_round_json)

[
  {
    "Year": 2000,
    "Round": 1,
    "Overall_Pick": 1,
    "Team": "Marlins",
    "Signed": "Y",
    "Bonus": 3000000.0,
    "Name": "Adrian Gonzalez",
    "Position": "1B",
    "WAR": 43.5,
    "G": 1929.0,
    "AB": 7139.0,
    "HR": 317.0,
    "BA": 0.287,
    "OPS": 0.843,
    "G1": 0.0,
    "W": 0.0,
    "L": 0.0,
    "ERA": 0.0,
    "WHIP": 0.0,
    "SV": 0.0,
    "Draft_Type": "HS",
    "Drafted_Out_Of": "Eastlake HS (Chula Vista, CA)",
    "Team_Made_Playoffs": 0
  },
  {
    "Year": 2000,
    "Round": 1,
    "Overall_Pick": 2,
    "Team": "Twins",
    "Signed": "Y",
    "Bonus": 2500000.0,
    "Name": "Adam Johnson",
    "Position": "RHP",
    "WAR": -1.1,
    "G": 1.0,
    "AB": 2.0,
    "HR": 0.0,
    "BA": 0.0,
    "OPS": 0.0,
    "G1": 9.0,
    "W": 1.0,
    "L": 3.0,
    "ERA": 10.25,
    "WHIP": 2.05,
    "SV": 0.0,
    "Draft_Type": "4Yr",
    "Drafted_Out_Of": "California State University, Fullerton (Fullerton, CA)",
    "Team_Made_Playoffs": 0
  },
  {
    "Ye

In [18]:
with open('first_round_json.json', "w") as out_file:
    out_file.write(json.dumps(first_round_dict))

In [19]:
# converting data to list
first_round_list = first_round_df.values.tolist()
print(first_round_list)

[[2000, 1, 1, 'Marlins', 'Y', 3000000.0, 'Adrian Gonzalez', '1B', 43.5, 1929.0, 7139.0, 317.0, 0.287, 0.843, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'HS', 'Eastlake HS (Chula Vista, CA)', 0], [2000, 1, 2, 'Twins', 'Y', 2500000.0, 'Adam Johnson', 'RHP', -1.1, 1.0, 2.0, 0.0, 0.0, 0.0, 9.0, 1.0, 3.0, 10.25, 2.05, 0.0, '4Yr', 'California State University, Fullerton (Fullerton, CA)', 0], [2000, 1, 3, 'Cubs', 'Y', 2750000.0, 'Luis Montanez', 'SS', -1.3, 129.0, 305.0, 5.0, 0.223, 0.586, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'HS', 'Coral Park HS (Miami, FL)', 0], [2000, 1, 4, 'Royals', 'Y', 2500000.0, 'Mike Stodolka', 'LHP', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'HS', 'Centennial HS (Corona, CA)', 0], [2000, 1, 5, 'Expos', 'Y', 2950000.0, 'Justin Wayne', 'RHP', -1.5, 26.0, 12.0, 0.0, 0.0, 0.0, 26.0, 5.0, 8.0, 6.13, 1.65, 0.0, '4Yr', 'Stanford University (Palo Alto, CA)', 0], [2000, 1, 6, 'Devil Rays', 'Y', 2250000.0, 'Rocco Baldelli', 'OF', 10.2, 519.0, 1910.0, 60.0, 0.278, 0.766, 0.0, 0.0

In [22]:
with open('first_round_list.js', "w") as out_file:
    out_file.write(json.dumps(first_round_list))